In [113]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from IPython.core.display import display, HTML
from functools import reduce
from sklearn import metrics

#display(HTML("<style>.container { width:100% !important; }</style>"))

import re
import random
import statistics
text = open("train.txt").readlines()
textLen = len(text)

In [114]:
#Decoupage du texte en cinq (k=5) sous parties
def textToPortions(text,k):
    portions=[]
    increments = round(textLen/k)
    for i in range(0,textLen,increments):
        portions.append((i,i+increments-1))
        
    return portions

In [115]:
def portionsToSubtext(textArray, portions):
    resText=[]
    for i,j in portions:
        for l in textArray[i:j+1]:
            resText.append(l)
            
    return resText

In [116]:
#Shuffle Text partitions
#Input number of (train, test, validation partitions) = 5 

def createShuffledSubText(originalText,train=None,test=None,validation=None):
    
    partitions = textToPortions(originalText,5)
    result = { 'Train-Test': [], 'validation': partitions.pop()}
    
    for i in range(len(partitions)):
        curr = {'train' : [partitions[i],partitions[(i+1)%4],partitions[(i+2)%4]], 'test' : [partitions[(i+3)%4]], 'language':{}, 'words':{}, 'score':0, 'failed':[]}
        result['Train-Test'].append(curr)
    return result

In [117]:
#Test
"""
portions = textToPortions(text,100)
processedText = portionsToSubtext(text,portions)
print("Test passed" if len(processedText) == len(text) else "Failed")
"""

'\nportions = textToPortions(text,100)\nprocessedText = portionsToSubtext(text,portions)\nprint("Test passed" if len(processedText) == len(text) else "Failed")\n'

In [118]:
Data = createShuffledSubText(text)
Train_Test = Data ['Train-Test']
Valid = Data['validation']

In [119]:
def tokenize(t):
    # Input t as a paragraph, returns:
    # list of (list of words,lenth of the list, and phrases count)
    
    t=t.lower()
    tkns = []

    phrases  = 0;
    for s in t.split("."):
        phrases +=1
        for w in s.split():
            tkns.append(w)
    return [tkns,len(tkns),phrases]

In [120]:
def wordCounter(row,language,TT):
    
    # Input the corpus as an array of inputs paragraphes, a target language, returns:
    # list of (length of words in a dictionary, [distinct words count, sentences count], the dictionary of words)
    
    dict = {}
    tk = tokenize(row)
    
    for w in tk[0]:
        
        # Processing the lang dictionary
        dict[w]= ((dict[w][0])+1,'Pure') if w in dict else (1,'Pure')
                                
        #Processing the global dictionary
        if w not in TT['words']: 
            TT['words'][w]={language:1}
            
        elif w in TT['words']:
            
            if language not in TT['words'][w]:
                TT['words'][w][language]= 1
            else:
                TT['words'][w][language]+=1       
                
    return [len(dict.keys()),tk[1],tk[2],dict,TT]

In [121]:
def addDict(d1, d2):
    for w in d2:
        if w not in d1:
            d1[w]=d2[w]
        else: 
            d1[w]=((d1[w][0]+d2[w][0]),d1[w][1])
    return d1

In [122]:
#Word Count   
for TT in Train_Test:
    
    train = portionsToSubtext(text,TT['train'])
    
    for i in range(len(train)):
        lang = train[i][1:4]
        wc = wordCounter(train[i][5:-1].lower(),lang,TT)
        TT=wc[4]

        if lang in TT['language']:

            TT['language'][lang]['Size'     ]+=   1             #Taille
            TT['language'][lang]['Distinct' ]+= wc[0]           #Unique Words
            TT['language'][lang]['Words'    ]+= wc[1]           #All words 
            TT['language'][lang]['Phrases'  ]+= wc[2]           #Sentences
            TT['language'][lang]['Avg'      ]+= 0               #Avg
            TT['language'][lang]['meanWords']+= 0        #Avg Word Length
            TT['language'][lang]['Dict'     ] = addDict(TT['language'][lang]['Dict'],wc[3])

        else :

            TT['language'][lang]= {'Size':1,'Distinct':wc[0],'Words':wc[1],'Phrases':wc[2], 'Avg':0 ,'meanWords':0,'Dict':wc[3]}

In [123]:
#Train_Test[0]

In [124]:
for TT in Train_Test:
    #Taille Moyenne des phrase en mots
    print("Partition", TT['train'],'\n')
    for l in TT['language']:
        TT['language'][l]['Avg']=round(TT['language'][l]['Words']/TT['language'][l]['Phrases'],1)

        print(l,"Total Words:",TT['language'][l]['Words'],"Distinct Words:",TT['language'][l]['Distinct'],"Total Phrases:",TT['language'][l]['Phrases'],"AVG Word/Phrase:",TT['language'][l]['Avg'])
    print("\n")

Partition [(0, 1979), (1980, 3959), (3960, 5939)] 

GER Total Words: 182772 Distinct Words: 84229 Total Phrases: 9274 AVG Word/Phrase: 19.7
TUR Total Words: 179743 Distinct Words: 81300 Total Phrases: 10097 AVG Word/Phrase: 17.8
CHI Total Words: 182279 Distinct Words: 81090 Total Phrases: 9881 AVG Word/Phrase: 18.4
TEL Total Words: 186605 Distinct Words: 84139 Total Phrases: 9608 AVG Word/Phrase: 19.4
ARA Total Words: 163522 Distinct Words: 75109 Total Phrases: 7185 AVG Word/Phrase: 22.8
SPA Total Words: 184894 Distinct Words: 82879 Total Phrases: 7817 AVG Word/Phrase: 23.7
HIN Total Words: 196553 Distinct Words: 91970 Total Phrases: 9929 AVG Word/Phrase: 19.8
JPN Total Words: 165382 Distinct Words: 73065 Total Phrases: 10157 AVG Word/Phrase: 16.3
KOR Total Words: 177133 Distinct Words: 78866 Total Phrases: 10660 AVG Word/Phrase: 16.6
FRE Total Words: 186696 Distinct Words: 85711 Total Phrases: 9378 AVG Word/Phrase: 19.9
ITA Total Words: 171516 Distinct Words: 80050 Total Phrases: 7424

In [125]:
def extract(a,b):
    
    for w in a:
        """
        if type(a[w])=='List' and a[w][1]=='Dirty':
            continue;
        if type(a[w])=='int' and w in b:
           
            a[w]=[a[w],'Dirty']         
        else :
            a[w]=[a[w],'Pure']
        """
        if a[w][1]=='Dirty':
            continue;
        elif w in b: 
            a[w]=(a[w][0],'Dirty')
    return a

In [126]:
for TT in Train_Test:
    
    langs = list(TT['language'].keys())
    size  = len(langs)

    for x in range(size):
        for y in range(size):
            if x != y:
                TT['language'][langs[x]]['D'] = extract(TT['language'][langs[x]]['Dict'],TT['language'][langs[y]]['Dict'])

In [127]:
Train_Test[0]['words']['the']
#Train_Test[0]['language']['GER']['Dict']['ithe']

{'GER': 8041,
 'TUR': 8661,
 'CHI': 9195,
 'TEL': 10541,
 'ARA': 7324,
 'SPA': 9064,
 'HIN': 9937,
 'JPN': 6518,
 'KOR': 6700,
 'FRE': 7858,
 'ITA': 8352}

In [128]:
for TT in Train_Test:
    #Testing acuracy of both dictionaries 
    for word in TT['words'].keys():
        for lang in TT['words'][word].keys():
            if word not in TT['language'][lang]['Dict']:
                print(TT['train'],word)
                break

In [129]:
def inter(l1, l2):
    dict = {}
    for e in l2:
        if e in l1:
            dict[e]=0
    return dict

In [107]:
def magicBox(t):
    
    words=tokenize(t)[0]
    langs = list(language.keys())
            
    LT    = len(words)
    stats = {}
    
    for lang in langs: 
        
        D = language[lang]['Dict']
        LD = len(D)
        score = 0

        for w in words:
            score += 1 if w in D and D[w][1]=='Pure'else 0
            
        stats[lang]=(score/LD)
    print(stats)
        
    return max(stats.keys(), key=(lambda k: stats[k]))

In [108]:
def box(t,TT,debug):
    
    tokens =tokenize(t)[0]
    
    hasSetPure   = False
    hasSetDirty  = False
    hasSetGlobal = False
    
    exequos = []
    
    langStats = {
        
        'FRE':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'ARA':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'CHI':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'TUR':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'TEL':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'GER':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'JPN':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'HIN':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'SPA':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'KOR':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } },
        'ITA':{ 'pure':0, 'unknown':0, 'global':0,'dirty':{'GER':0, 'TEL':0, 'SPA':0, 'KOR':0, 'JPN':0, 'ARA':0, 'FRE':0, 'HIN':0, 'TUR':0, 'CHI':0, 'ITA':0 } }
    }
     
    for each in list(langStats.keys()):
        
        D = TT['language'][each]['Dict']
        for w in tokens:
            
            if w in D: 
                
                if D[w][1]=='Pure':
                    langStats[each]['pure']+=round(1/len(D),7)
                    hasSetPure=True;
                    
                elif D[w][1]=='Dirty':
                    for e in TT['words'][w].keys():
                        langStats[each]['dirty'][e]+=TT['words'][w][e]
                        hasSetDirty=True;
                        
            elif w in TT['words']:
                langStats[each]['global']+=round(1/len(TT['words']), 7)
                hasSetGloabl=True;
            else :
                langStats[each]['unknown']+=1
                
        if debug :
            print(each,langStats[each])
        
    #Traitement des globaux_excluded       
    if not hasSetPure:
        return min(langStats.keys(), key=(lambda k: langStats[k]['global']))
    else :
        return max(langStats.keys(), key=(lambda k: langStats[k]['pure']))

In [105]:
def score (TT): 
    
    Test = portionsToSubtext(text,TT['test'])
    acc = 0
    failed = []
    L = len(Test)
    for i in range(L):

        T=Test[i]
        trueLang = T[1:4]

        predictedLang = box(T[5:-1],TT,False)
        if predictedLang != trueLang:
            #print(i,trueLang,predictedLang)
            failed.append(i)

        if predictedLang == trueLang:
            acc+=1/L
    return acc,failed

In [106]:
for TT in Train_Test:
    TT['score'],TT['failed']=score(TT)

In [86]:
i = 1
for each in Train_Test:
    print("Portion",i, 'score',round(each['score']*100,2),'%')
    i+=1

Portion 1 score 25.56 %
Portion 2 score 26.46 %
Portion 3 score 24.75 %
Portion 4 score 25.66 %


In [77]:
i = 1
for each in Train_Test:
    print("Portion",i, 'score',round(each['score']*100,2),'%')
    i+=1

Portion 1 score 30.15 %
Portion 2 score 30.45 %
Portion 3 score 29.04 %
Portion 4 score 29.8 %


In [156]:
#Analysing failed
"""for i in Train_Test[0]['failed']:
    Text = portionsToSubtext(text,Train_Test[0]['test'])
    box(Text[i],Train_Test[0],True)
    print('--')
"""

"for i in Train_Test[0]['failed']:\n    Text = portionsToSubtext(text,Train_Test[0]['test'])\n    box(Text[i],Train_Test[0],True)\n    print('--')\n"

In [155]:
def createFeature(T,TT):
    tk = tokenize(T[5:-1])
    return [len(TT['language'][T[1:4]]['Dict']),len(t),tk[1], tk[2],(tk[1]/tk[2])]

In [152]:
#len(TT['language'][T[1:4]]['Dict'])

In [153]:
#Random Forest Classification with Cross Validation
for TT in Train_Test:
    
    Train = portionsToSubtext(text,TT['train'])
    Test  = portionsToSubtext(text,TT['test' ])
    
    X_train =[] 
    y_train =[]

    for t in Train:
        y_train.append(t[1:4])
        X_train.append(createFeature(t,TT))
        
    X_test =[] 
    y_test =[]

    for t in Test:
        y_test.append(t[1:4])
        X_test.append(createFeature(t,TT))

    clf=RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,y_train)
    
    y_pred=clf.predict(X_test)
    TT['score']=metrics.accuracy_score(y_test, y_pred)

In [154]:
i = 1
for each in Train_Test:
    print("Random Forest Classification with Cross Validation Portion",i, 'score',round(each['score']*100,2),'%')
    i+=1

Random Forest Classification with Cross Validation Portion 1 score 14.14 %
Random Forest Classification with Cross Validation Portion 2 score 13.79 %
Random Forest Classification with Cross Validation Portion 3 score 13.08 %
Random Forest Classification with Cross Validation Portion 4 score 14.19 %


In [638]:
langues = ['FRE','ARA','CHI','TUR','TEL','GER','JPN','HIN','SPA','KOR','ITA']
def langue(texte):
    return texte[1:4]

In [639]:
def boite_vide(mot):
    return (mot,
           {'ALL' : 11,
            'GER' : 1,
            'TUR' : 1,
            'CHI' : 1,
            'TEL' : 1,
            'ARA' : 1,
            'SPA' : 1,
            'HIN' : 1,
            'JPN' : 1,
            'KOR' : 1,
            'FRE' : 1,
            'ITA' : 1},
           {'ALL' : 11,
            'GER' : 1,
            'TUR' : 1,
            'CHI' : 1,
            'TEL' : 1,
            'ARA' : 1,
            'SPA' : 1,
            'HIN' : 1,
            'JPN' : 1,
            'KOR' : 1,
            'FRE' : 1,
            'ITA' : 1},
           )

In [640]:
def entraine(boite,texte,langue):
    mot,present,absent = boite
    if mot in texte :
        present['ALL']  = present['ALL']  + 1
        present[langue] = present[langue] + 1
    else :
        absent['ALL']   = absent['ALL']  + 1
        absent[langue]  = absent[langue] + 1

In [641]:
def evalue(boite,texte):
    mot,present,absent = boite
    if mot in texte :
        return (True,present,present['ALL'])
    else:
        return (False,absent,present['ALL'])

In [642]:
def cree_boites(mots):
    boites = []
    for mot in mots:
        boites.append(boite_vide(mot))
    return boites

In [643]:
def entraine_boites(boites,textes):
    for texte in textes :
        texte = tokenize(text[5:-1])[0]
        l = langue(texte)
        for boite in boite :
            entraine(boite,texte,l)

In [157]:
def cree_boites_entrainees(mots,textes):
    #creation des boites vides
    boites = {}
    for mot in list(mots):
        boites[mot] = boite_vide(mot)
    
    #ajout des mots «présents»
    nb = {'GER' : 0,'TUR' : 0,'CHI' : 0,'TEL' : 0,'ARA' : 0,'SPA' : 0,'HIN' : 0,'JPN' : 0,'KOR' : 0,'FRE' : 0,'ITA' : 0}
    for texte in textes :
        langue = texte[1:4]
        nb[langue] += 1
        mots = set()
        mots.update(tokenize(texte[5:-1])[0])
        for mot in mots :
            entraine(boites[mot],[mot],langue)
            
    #evaluation des mots «absents»
    for boite in boites.values() :
        mot,present,absent = boite
        absent['ALL'] = len(textes) - present['ALL']
        for langue in langues :
            absent[langue] = nb[langue] - present[langue]
    
    return boites

In [645]:
def f(acc,resultat,simple):
    s,v,m = resultat
    if simple or s:
        for i in langues :
            acc[i] += v[i]*m
def evaluer(boites,texte,simple = True,f = f):
    acc = {'GER' : 0,'TUR' : 0,'CHI' : 0,'TEL' : 0,'ARA' : 0,'SPA' : 0,'HIN' : 0,'JPN' : 0,'KOR' : 0,'FRE' : 0,'ITA' : 0}
    for boite in boites :
        f(acc,evalue(boite,texte),simple)
    m = 0
    langue = None
    for l in list(acc.keys()):
        if acc[l] > m :
            m = acc[l]
            langue = l
    return langue

In [665]:
def f(acc,resultat,simple):
    s,v,m = resultat
    if simple or s:
        for i in langues :
            acc[i] += (v[i])
def evaluer(boites,texte,simple = True,f = f):
    init = 1
    acc = {'GER' : init,'TUR' : init,'CHI' : init,'TEL' : init,'ARA' : init,'SPA' : init,'HIN' : init,'JPN' : init,'KOR' : init,'FRE' : init,'ITA' : init}
    for boite in boites :
        f(acc,evalue(boite,texte),simple)
    m = 0
    langue = None
    for l in list(acc.keys()):
        if acc[l] > m :
            m = acc[l]
            langue = l
    return langue

In [666]:
boites = cree_boites_entrainees(list(words.keys()),text)

In [667]:
evaluer(list(boites.values()),tokenize(text[0][5:-1])[0],True)

'JPN'

In [668]:
def score(boites,texte = text) :
    s = 0
    for t in texte :
        if evaluer(boites,tokenize(t[5:-1])[0]) == langue(t):
            s += 1
    return s/len(texte)

In [669]:
score(list(boites.values()),text[1:100])

0.1111111111111111

In [ ]:
Y|Avg Word/Phrases|Total_Words|Distinct_Words|Total_Phrases|AVG Word/Phrase|pure_count|dirty count|globaux_excluded|'dirty': {'GER': 295926, 'TEL': 266660, 'SPA': 319469, 'KOR': 264377, 'JPN': 251467, 'ARA': 257221, 'FRE': 317691, 'HIN': 289154, 'TUR': 280723, 'CHI': 301288, 'ITA': 284233}|

In [ ]:
def export(predict, corpus_path):
    
    export = open("Charles & Valery - predicted_"+corpus_path, "wt")
    file   = open(corpus_file_path,"r+")
    
    for line in file:
        export.write('('+ (predict(line[5:-1]))+')'+line[5:-1])
        
    export.close()
    file.close()